Top2Vec 是一种用于主题建模和语义搜索的算法。它自动检测文本中存在的主题并生成联合嵌入的主题、文档和词向量。训练 Top2Vec 模型后，您可以：

- 获取检测到的主题数。
- 获取话题。
- 获取主题大小。
- 获取分层主题。
- 按关键字搜索主题。
- 按主题搜索文档。
- 按关键字搜索文档。
- 找出相似的词。
- 查找类似的文档。
- 使用 RESTful-Top2Vec 公开模型
- 有关其工作原理的更多详细信息，请参阅论文。

**亮点**

- 自动查找主题数。
- 不需要停用词列表。
- 不需要词干/词形还原。
- 适用于短文本。
- 创建联合嵌入的主题、文档和词向量。
- 内置搜索功能。


**它是如何工作的？**

该算法做出的假设是，许多语义相似的文档都表明了一个潜在的主题。

第一步是创建文档和词向量的联合嵌入。一旦文档和单词被嵌入到一个向量空间中，算法的目标就是找到密集的文档集群，然后确定哪些单词将这些文档吸引到一起。每个密集区域是一个主题，将文档吸引到密集区域的词就是主题词。

In [ ]:
!pip3 install top2vec

## 1. 导入数据
使用某灾难数据集，这里是存在标注的标签，但是我们假设不用label的，仅作为评判Top2vec运行效果的标准。

In [1]:
from IPython.display import display
from cntext import STOPWORDS_zh
from top2vec import Top2Vec
import pandas as pd
import jieba

df = pd.read_csv('cnews.csv')
df.head()

,label,text
0,教育,澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...
1,体育,快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...
2,科技,3英寸屏高清闪存DV 三洋TH1特价1499 作者：中关村在线 飘雪 ...
3,时尚,贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...
4,房产,三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...


In [2]:
df.label.value_counts()

时政    120
科技    106
时尚    106
财经    105
家居    103
教育     97
娱乐     96
体育     95
房产     87
游戏     85
Name: label, dtype: int64

## 2. 清洗数据
一般而言，作中文文本分析，需要把中文分词构造成类西方语言(空格间隔词语的文本)风格。在此期间，顺便将停用词剔除。其实在用top2vec时，不剔除停用词影响也不大。

In [3]:
def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in STOPWORDS_zh]
    return ' '.join(words)


df['cleantext'] = df.text.apply(clean_text)
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sc/3mnt5tgs419_hk7s16gq61p80000gn/T/jieba.cache
Loading model cost 0.682 seconds.
Prefix dict has been built successfully.


,label,text,cleantext
0,教育,澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...,澳 移民 子女 成长 记 带 中国 心 融入 主流 社会 新华网 悉尼 5 31 日电 ...
1,体育,快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...,快船 vs 火箭 首发 休城 旨在 练兵 小德 帕特森 进先发 新浪 体育讯 北京 时间...
2,科技,3英寸屏高清闪存DV 三洋TH1特价1499 作者：中关村在线 飘雪 ...,3 英寸 屏 高清 闪存 DV 三洋 TH1 特价 1499 ...
3,时尚,贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...,贝嫂 乏味 乏味 买账 Victoria Beckham 乏味 乏味 买账 ...
4,房产,三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...,三亚 岭南 赶房集 金九银 十再兴 购房 游 纯粹 旅行 闲适 有余 “ 收获 ” 不足...


## 3. 训练模型
Top2vec有一下四个常用参数

**Top2vec(documents, min_count, speed, workers)**
- documents: 文档列表
- min_count: 词语最少出现次数。低于min_count的词不加入模型中
- speed: 训练速度，参数默认"learn"
   - "fast-learn"  速度最快，训练效果最差
   - "learn"       速度，训练效果中等
   - "deep-learn"  速度最慢，训练效果最佳
- workers: 并行运行数，该值最大取值为电脑CPU的核数。

In [4]:
model = Top2Vec(documents=df['cleantext'].to_list(), 
                min_count=10,
                speed="deep-learn",  
                workers=8)

2021-12-14 20:21:10,318 - top2vec - INFO - Pre-processing documents for training
2021-12-14 20:21:10,871 - top2vec - INFO - Creating joint document/word embedding
2021-12-14 20:25:06,082 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-14 20:25:14,645 - top2vec - INFO - Finding dense areas of documents
2021-12-14 20:25:14,683 - top2vec - INFO - Finding topics


In [5]:
# 话题个数
model.get_num_topics()

11

In [8]:
# 各话题数量
topic_sizes, topic_nums = model.get_topic_sizes()

{"topic_sizes":topic_sizes, 
 "topic_ids":topic_nums}

{'topic_sizes': array([116, 108, 105, 104, 100,  97,  95,  94,  89,  74,  18]),
 'topic_ids': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])}

## 4. get_topics
用pyecharts词云图显示**话题信息**， 为了简化代码，将该功能封装为函数

In [9]:
def gen_wordcloud(topic_words, word_scores, topic_id):
    """
    topic_words: 主题词列表
    word_scores: 主题特征词的权重得分(词语表征主题的能力)
    topic_id: 主题id
    """
    import pyecharts.options as opts
    from pyecharts.charts import WordCloud
    from IPython.display import display
    
    data = [(word, str(weight)) for word, weight in zip(topic_words, word_scores)]

    wc = WordCloud()
    wc.add(series_name="", data_pair=data, word_size_range=[6, 88])
    wc.set_global_opts(
        title_opts=opts.TitleOpts(title="Topic_{topic_id}".format(topic_id=topic_id), 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=23)),
        tooltip_opts=opts.TooltipOpts(is_show=True))

    display(wc.render_notebook())

In [11]:
topic_wordss, word_scoress, topic_ids = model.get_topics(11)

for topic_words, word_scores, topic_id in zip(topic_wordss, word_scoress, topic_ids):
    gen_wordcloud(topic_words, word_scores, topic_id)

## 5. get_documents_topics
get_documents_topics(doc_ids, num_topics=1)
- doc_ids: 待查询文档id列表
- num_topics: 返回某文档可能归属话题的个数

In [40]:
df.head()

,label,text,cleantext
0,教育,澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...,澳 移民 子女 成长 记 带 中国 心 融入 主流 社会 新华网 悉尼 5 31 日电 ...
1,体育,快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...,快船 vs 火箭 首发 休城 旨在 练兵 小德 帕特森 进先发 新浪 体育讯 北京 时间...
2,科技,3英寸屏高清闪存DV 三洋TH1特价1499 作者：中关村在线 飘雪 ...,3 英寸 屏 高清 闪存 DV 三洋 TH1 特价 1499 ...
3,时尚,贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...,贝嫂 乏味 乏味 买账 Victoria Beckham 乏味 乏味 买账 ...
4,房产,三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...,三亚 岭南 赶房集 金九银 十再兴 购房 游 纯粹 旅行 闲适 有余 “ 收获 ” 不足...


In [48]:
# 查第一条文档的
model.get_documents_topics(doc_ids=[0], num_topics=1)

(array([9]),
 array([0.13393445], dtype=float32),
 array([['留学', '学生', '申请', '签证', '大学', '学费', '本科', '名校', '留学生', '加拿大',
         '课程', '申请者', '院校', '学校', '移民', '英国', '高等教育', '教育', '录取', '高中生',
         '出国', '人数', '高中', '学位', '就读', '生活费', '家长', '澳大利亚', '申请人', '预科',
         '美国', '该国', '新政', '招生', '文凭', '学士学位', '公立', '私立', '攻读', '高校',
         '绿卡', '升学', '学历', '雅思', '入学', '资助', '移民部', '技术移民', '万至', '学期']],
       dtype='<U9'),
 array([[0.6926953 , 0.6242467 , 0.5911637 , 0.5618203 , 0.54714125,
         0.54360884, 0.5397957 , 0.5385162 , 0.53832144, 0.5268167 ,
         0.5255044 , 0.52232724, 0.52129036, 0.5149227 , 0.49912092,
         0.46216848, 0.46023563, 0.45987687, 0.45913237, 0.45762828,
         0.45715228, 0.45121962, 0.45040804, 0.44802257, 0.44495288,
         0.44481775, 0.44124833, 0.43432242, 0.43391675, 0.43311685,
         0.4321192 , 0.4318304 , 0.43135592, 0.42906424, 0.42849895,
         0.42731148, 0.42642018, 0.420944  , 0.417553  , 0.41722208,
         0.4112

## 6. search_topics
根据关键词搜索话题，查某词是否属于某话题，属于该主题的概率
search_topics(keywords, num_topics, keywords_neg=None)
- keywords: 关键词列表
- num_topics: 返回话题个数，按照语义相似度从高到低排序
- keywords_neg: 反义词列表

In [12]:
def gen_wordcloud2(query_word, topic_words, word_scores, topic_id, topic_probability):
    """
    query_word: 待查询词
    topic_words: 主题词列表
    word_scores: 主题特征词的权重得分(词语表征主题的能力)
    topic_id: 主题id
    topic_probability: 主题概率
    """
    import pyecharts.options as opts
    from pyecharts.charts import WordCloud
    from IPython.display import display
    
    data = [(word, str(weight)) for word, weight in zip(topic_words, word_scores)]

    wc = WordCloud()
    wc.add(series_name="", data_pair=data, word_size_range=[6, 88])
    title = """Word{query_word}\nTopic_{topic_id}\nProbability:{probability:.2f}""".format(query_word=query_word,
                                                              topic_id=topic_id, 
                                                              probability=topic_probability)
    wc.set_global_opts(
        title_opts=opts.TitleOpts(title=title, 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
        tooltip_opts=opts.TooltipOpts(is_show=True))

    display(wc.render_notebook())

In [15]:
query_word = "电影"
topic_wordss, word_scoress, topic_scores, topic_ids = model.search_topics(keywords=[query_word], num_topics=4)
for topic_words, word_scores, topic_score, topic_id in zip(topic_wordss, word_scoress, topic_scores, topic_ids):
    if topic_score>0.5:
        gen_wordcloud2(query_word=query_word, 
                       topic_words=topic_words, 
                       word_scores=word_scores, 
                       topic_id=topic_id, topic_probability=topic_score)

## 7. query_topics
根据一段文本寻找最符合该文本的话题
query_topics(query, num_topics)
- query: 查询文本，注意是用空格间隔词语的文本
- num_topics: 返回的话题数

返回话题特征词列表， 话题特征词权重， 话题概率， 话题id

In [39]:
querytext = '刘晓庆 55 岁 近日 颁奖礼 刘晓庆 一袭 宝蓝色 超低 胸 V 领 长裙 亮相 轻薄 蕾丝 奢华 皮草 艳丽 色彩 翠绿'
topic_words, word_scores, topic_scores, topic_ids = model.query_topics(query=querytext, 
                                                                       num_topics=2)
print('可能归属的话题有: ', topic_ids)
print('归属于该话题的概率', topic_scores)

可能归属的话题有:  [1 4]
归属于该话题的概率 [0.3688392  0.21418373]


## 8. search_documents_by_keywords

根据关键词，筛选文档

search_documents_by_keywords(keywords, 
                             num_docs, 
                             keywords_neg=None, 
                             return_documents=True)

In [30]:
#文档， 语义相关性， 文档id
docs, scores, doc_ids = model.search_documents_by_keywords(keywords=['搭配'], 
                                                         num_docs=3, 
                                                         keywords_neg=None, 
                                                         return_documents=True)
for doc, score, doc_id in zip(docs, scores, doc_ids):
    print(f"Document: {doc_id}, Semantic similarity: {score}")
    print(doc)
    print('----------')
    print()

Document: 870, Semantic similarity: 0.46936729550361633
组图 看达人 演绎 豹纹 军装 风 导语 懂得 潮流 总是 知道 适合 今冬 流行 亮点 太 军装 豹纹 类似 民族风情 想要 知道 搭配 快 看看 时尚 达 穿 军绿色 宽松 款 大衣 不失 俏皮 味道 高腰 设计 短裙 有效 提升 腰线 衬托出 修长 美腿 豹纹 今年 冬季 抢眼 搭配 元素 加上 驼色 针织衫 灰色 围巾 暖 棕色 手 挎包 整体 色调 统一 迷人 棕色 蓝色 结合能 眼前一亮 简洁 款式 依然 突显 独特 品味 宽松 针织 外套 衬托出 优美 身形 搭配 同样 沉闷 黑色 包包 性感 丝袜 装扮 依然 透露 出 迷人 气息 立领 衬衫 加上 深黄 高腰 裤 摩登 感 十足 随意 披上 外套 更显 慵懒 个性 法式 风情
----------

Document: 450, Semantic similarity: 0.46842482686042786
街 拍 爱 招摇过市 毛茸茸 ( 组图 ) 导语 皮草 每个 冬天 可能 丢弃 每个 需要 温暖 早些 相比 人造皮 草比 真皮 草 风头 更劲 时尚 环保 大牌 秀 场上 超模 一个个 穿着 人造皮 草 “ 招摇过市 ” 之后 街头 潮人 没有 理由 拒绝 外形 酷酷 这件 气场 皮草 单品 配合默契 摇滚 风 配饰 搭配 黑色 皮草 长 背心 更显 利落 酷酷 黑色 皮草 搭配 蓝色 衬衣 不同 感觉 加上 下半身 底裤 时髦 包包 颜色 提亮 整身 装扮 抹胸 式 皮草 特点 高贵典雅 适合 搭配 连衣裙 装饰 增添 时尚 美感 复古 圆点 连衣裙 搭配 宽松 棕色 皮草 衣 名媛 感觉 典雅 淑女 短款 黑色 皮草 搭配 贴身 仔裤 搭配 长靴 潇洒 帅气 茸茸 帽子 增添 不少 甜美 感
----------

Document: 665, Semantic similarity: 0.45364660024642944
组图 韩国 明星 街 拍 各显 本色 魅力 导语 很多 迷 韩剧 剧中 女星 逐渐 变成 家喻户晓 红人 韩 剧中 靓丽 女星 示范 想 靓丽 成为 众人 瞩目 焦点 摆脱 荧幕 光环 现实 中是 李孝利 韩国 有人 气 女歌手 走红 速度 之快 成为 影

## 9. search_documents_by_topic
根据指定的topic_id， 显示该主题前num_docs个文档，显示的文档是根据概率从高到低降序显示

In [31]:
#查看topic4的前5条文档
topic_id = 4
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=topic_id, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Semantic similarity: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 91, Semantic similarity: 0.4055505692958832
-----------
遍地 狼烟 场面 恢弘   打造 精良 狙击枪 战 类型 片 新浪 娱乐 讯   电影 遍地 狼烟 12 2 上映 该片 何润东 ( 微博 ) 宋佳 ( 微博 ) 梁家辉 何晟铭 ( 微博 ) 主演 展现 狙击手 不为人知 一面 何润东 日前 透露 拍摄 该片 苦练 身材 希望 电影 能够 看起来 “ 够 爷们 ” 狙击 题材 枪战 片 遍地 狼烟 取材 狙击手 创作 初衷 导演 胡 大为 表示 “ 狙击手 生活 远远 不像 想像 简单 辛苦 危险 曾经 一位 狙击手 朋友 等待 目标 潜伏 树上 达 一周 久 枪法 百发百中 之外 狙击手 需要 经受 常人 难以忍受 痛苦 ” 希望 借由 遍地 狼烟 狙击手 不为人知 一面 呈现 观众 面前 何润东 苦练 身材 何润东 片中 饰演 “ 枪神 ” 牧良逢 透露 显得 “ 够 爷们 ” 接到 拍摄 任务 不断 加大 健身 力度 “ 每天 片场 第一件 事 几位 男 主演 ‘ 肌肉 ’ 相信 每个 男人 从小 希望 顶天立地 电影 里 角色 成为 一个 罕见 神枪手 体验 永世难忘 绝美 爱情 ” 原来 片中 何润东 宋佳 饰演 柳烟 展开 一段 生死 绝恋 特效 堪比 好莱坞 据悉 遍地 狼烟 战斗 场面 拍摄 使用 肩 扛 手提 摄像机 剪接 快速 特效 制作 国际 特效 团队 MACROGRAPH 担纲 导演 胡 大为 表示 “ 遍地 狼烟 男人 爆发力 好莱坞 暑期 大片 视听 轰炸 方面 带给 极致 快感 ” 主演 何润东 忍不住 夸赞 “ 前一阵子 剪接室 完整 遍地 狼烟 真的 看到 缺氧 刺激 战争片 一部 纯 狙击手 电影 ”
-----------

Document: 392, Semantic similarity: 0.38211899995803833
-----------
组图 大S 新片 腿 夹 色诱 古天乐   动作 搞笑 情色 新浪 娱乐 讯   大S 古天乐 结戏缘 电影 大内 密探 灵灵狗 饰演 夫妻 使出 浑身解数 色诱 古天乐 露出 性感 香肩 双脚 夹紧 画面 相当 搞笑 剧中 打斗 戏 不断 牺牲 形象 引诱 大S 挑战 笑 非常 紧张 平常

In [33]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["搭配", "高跟鞋"], num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Semantic similarity: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 727, Semantic similarity: 0.5883481502532959
-----------
组图 冷气 办公室   连衣裙 配小 坎肩 美国 设计师 Diane   Von   Furstenberg 曾经 感觉 女人 穿 连衣裙 女人 找到 一件 适合 dream   dress 重要 无需 费神 搭配 单穿 连身 优雅 飞扬 裙摆 似乎 告诉 女 连衣裙 玩起 High   Fashion 变脸 游戏 DKNY 绿色 连衣裙   新品 未 定价 H & M 黑色 外套   新品 未 定价 Agatha 配件 新品 未 定价 C . Banner 高跟鞋   新品 未 定价 低 V 领 连衣裙 秀出 属于 性感 更好 展现出 颈部 线条 搭配 修身 剪裁 西装 短款 皮手套 极具 欧美 明星 范儿 细 高跟鞋 更好 突出 双腿 长度 整体 显得 轻盈 不少 On & on 米色 连衣裙   新品 未 定价 Asobio 针织 外套   RMB   449 Kookai 金色 腰带 Jc   collezione 高跟鞋   RMB   1198 H & M 手镯   新品 未 定价 不想 格纹 图案 穿 满身 披 一件 菱形 针织衫 原本 保守 连衣裙 增加 亮点 聪明 选择 金属 色系 腰带 视觉 拉长 身材 比例 衬托出 女性 纤腰 美态 曲线 身材 令人 气 飙升 一级 DKNY   黑色 连衣裙 Miss   Sixty   条纹 T恤   RMB   850 Jc   collezione 高跟鞋   RMB   828Kookai 草帽   新品 未 定价 Agatha 项链   新品 未 定价 Miss   Sixty 手包   新品 未 定价 裙摆 雕塑 般 造型 十足 连衣裙 TEE 叠 穿 出上 紧下松 层次感 不同 面料 产生 质感 软硬 区别 背心 款式 带点 俏丽 不会 显得 过于 成熟 深邃 显瘦 黑色 每个 穿 明艳 动人 W . Doubleudot 粗 连衣裙   新品 未 定价 Marella 粗 外套   新品 未 定价 Jc   collezione 高跟鞋   RMB   898 Max   Mara 蓝色 手 包   新品 未 定价 面料 拼接 运用 简单 款式 看起来 丰富 有

## 10. get_topic_hierarchy
对话题进行分类，需要
1. 先执行model.hierarchical_topic_reduction
2. 再执行model.get_topic_hierarchy。

In [21]:
# 将话题分为2类
model.hierarchical_topic_reduction(num_topics=2)
model.get_topic_hierarchy()

[[6, 4, 1, 3, 8, 5], [7, 2, 10, 9, 0]]

## 11. similar_words
查找相似词， 该方法其实也可以用于扩充词典。


similar_words(keywords, num_words, keywords_neg=None)
- keywords: 待查询关键词列表
- num_words: 返回相似词个数
- keywords_neg: 指定反义词列表

In [22]:
# 查找【增进】的最相似的10个词
model.similar_words(keywords=["增进"], 
                    num_words=10, 
                    keywords_neg=None)

(array(['两国关系', '王刚', '友好', '会见', '两国', '东盟', '温家宝', '中美', '伙伴', '双方'],
       dtype='<U4'),
 array([0.5130112 , 0.50116096, 0.47922852, 0.46926211, 0.46253758,
        0.4395747 , 0.43768376, 0.42893141, 0.41960889, 0.41416299]))

## 12. save
训练不易， 记得保存模型。

In [49]:
model.save('随便起个名字.pkl')